In [1]:
reviews = [
    # Review 1: Positive
    "I absolutely love the new QuantumX Pro camera! The picture quality is stellar and the battery life is amazing. Shipped super fast too. A++!",

    # Review 2: Negative with specific issue
    "The SonicWave earbuds have a serious design flaw. The left earbud stopped charging after just one week. I expected better for the price. Very disappointed.",

    # Review 3: Mixed with a question
    "The Titan smartwatch is decent. The screen is bright and the features are good, but the step counter seems inaccurate. It's off by at least 20%. Is there a way to calibrate it?",

    # Review 4: Negative with multiple issues
    "My order for the AeroDrone was a disaster. It arrived with a broken propeller and the battery was completely dead on arrival. Customer service has been unresponsive for 3 days.",

    # Review 5: Positive but mentions a minor issue
    "Overall, I'm happy with the PureGlow Air Purifier. It's quiet and effective. My only complaint is that the replacement filters are a bit expensive."
]

In [2]:
from transformers import AutoTokenizer

# Load GPT-2 tokenizer
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Load BERT-base-uncased tokenizer
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Example usage
text = "Hello, how are you?"

print("GPT-2 Tokenized:", gpt2_tokenizer.tokenize(reviews[3]))
print("BERT Tokenized:", bert_tokenizer.tokenize(reviews[3]))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

GPT-2 Tokenized: ['My', 'Ġorder', 'Ġfor', 'Ġthe', 'ĠAero', 'Dr', 'one', 'Ġwas', 'Ġa', 'Ġdisaster', '.', 'ĠIt', 'Ġarrived', 'Ġwith', 'Ġa', 'Ġbroken', 'Ġprope', 'ller', 'Ġand', 'Ġthe', 'Ġbattery', 'Ġwas', 'Ġcompletely', 'Ġdead', 'Ġon', 'Ġarrival', '.', 'ĠCustomer', 'Ġservice', 'Ġhas', 'Ġbeen', 'Ġun', 'responsive', 'Ġfor', 'Ġ3', 'Ġdays', '.']
BERT Tokenized: ['my', 'order', 'for', 'the', 'aero', '##dron', '##e', 'was', 'a', 'disaster', '.', 'it', 'arrived', 'with', 'a', 'broken', 'propeller', 'and', 'the', 'battery', 'was', 'completely', 'dead', 'on', 'arrival', '.', 'customer', 'service', 'has', 'been', 'un', '##res', '##pon', '##sive', 'for', '3', 'days', '.']
